In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input/self'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as npimg
import os

## Keras
import keras
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.callbacks
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense

import cv2
import pandas as pd
import random
import ntpath
import shutil

## Sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from imgaug import augmenters as aug

In [ ]:
TRAIN_TRACK_1 = True #True for track 1 

In [ ]:
## Store data
datadir = '../input/datasetv3/DatasetV3/DatasetV3'
imgdir = '../input/datasetv3/DatasetV3/DatasetV3/IMG'
columns = ['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed']
data = pd.read_csv(os.path.join(datadir, 'driving_log.csv'), names = columns)
pd.set_option('display.max_colwidth', -1)
data.head()

In [ ]:
def path_leaf(path):
  head, tail = ntpath.split(path)
  return tail

## Remove path of images
data['center'] = data['center'].apply(path_leaf)
data['left'] = data['left'].apply(path_leaf)
data['right'] = data['right'].apply(path_leaf)
data.shape

In [ ]:
from datetime import datetime

    #center_2022_05_15_21_00_09_631.jpg FIRST TRACK 2
if (TRAIN_TRACK_1):
    test_list = data['center'].tolist()
    boolean_list = []
    first_track2 = datetime(2022,5,15,21,0,9)
    print(first_track2)

    for index in range(len(test_list)):
        test_list[index] = test_list[index][7:-8].replace('_',' ')
        test_list[index] = datetime.strptime(test_list[index], '%Y %m %d %H %M %S')
        if(test_list[index] <= first_track2):
            boolean_list.append(True)
        else:
            boolean_list.append(False)
        
    data = (data[boolean_list])
    data.reset_index(drop=True, inplace=True)
data.head()

In [ ]:
## Visualize data
num_bins = 31
samples_per_bin = 1250
hist, bins = np.histogram(data['steering'], num_bins)
center = (bins[:-1] + bins[1:]) * 0.5  # center the bins to 0

## Plot
plt.bar(center, hist, width=0.05)
plt.plot((np.min(data['steering']), np.max(data['steering'])), (samples_per_bin, samples_per_bin))

In [ ]:
print('Total data: {0}'.format(len(data)))

## Make list of indices to remove
remove_list = []
for j in range(num_bins):
  list_ = []
  for i in range(len(data['steering'])):
    steering_angle = data['steering'][i]
    if steering_angle >= bins[j] and steering_angle <= bins[j+1]:
      list_.append(i)
  list_ = shuffle(list_)
  list_ = list_[samples_per_bin:]
  remove_list.extend(list_)
  
## Remove from extras from list
data.drop(data.index[remove_list], inplace=True)
print('Removed: {0}'.format(len(remove_list)))
print('Remaining: {0}'.format(len(data)))

## Plot
hist, _ = np.histogram(data['steering'], (num_bins))
plt.bar(center, hist, width=0.05)
plt.plot((np.min(data['steering']), np.max(data['steering'])), (samples_per_bin, samples_per_bin))

In [ ]:
def load_img_steering(datadir, df):
  image_path = []
  steering = []
  for i in range(len(data)):
    indexed_data = data.iloc[i]
    center, left, right = indexed_data[0], indexed_data[1], indexed_data[2]
    image_path.append(os.path.join(imgdir, center.strip()))
    steering.append(float(indexed_data[3]))
  image_paths = np.asarray(image_path)
  steerings = np.asarray(steering)
  return image_paths, steerings

image_paths, steerings = load_img_steering(datadir + '/IMG', data)


In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(image_paths, steerings, test_size=0.2, random_state=42, shuffle=True)

## Check that data is valid
print("Training Samples: {}\nValid Samples: {}".format(len(X_train), len(X_valid)))
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].hist(Y_train, bins=num_bins, width=0.05, color='blue')
axes[0].set_title('Training set')
axes[1].hist(Y_valid, bins=num_bins, width=0.05, color='red')
axes[1].set_title('Validation set')

In [ ]:
def augment_image(path, steering_angle):
    img = npimg.imread(path)
    if np.random.rand() < 0.5:
        pan = aug.Affine(translate_percent={'x':(-0.1,0.1),'y':(-0.1,0.1)})
        img = pan.augment_image(img)
    if np.random.rand() < 0.5:
        zoom = aug.Affine(scale=(1,1.2))
        img = zoom.augment_image(img)
    if np.random.rand() < 0.5:
        brightness = aug.Multiply((0.4,1.2))
        img = brightness.augment_image(img)
    if np.random.rand() < 0.5:
        img = cv2.flip(img,1)
        steering_angle *= -1
    
    return img, steering_angle
    

In [ ]:
def img_preprocess(img):
  #img = npimg.imread(img)
  
  ## Crop image to remove unnecessary features
  img = img[60:135, :, :]
  
  ## Change to YUV image
  img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
  
  ## Gaussian blur
  img = cv2.GaussianBlur(img, (3, 3), 0)
  
  ## Decrease size for easier processing
  img = cv2.resize(img, (200, 66))
  
  ## Normalize values
  img = img / 255
  return img

In [ ]:
def generate_batch(image_paths, steerings, batch_size, training_flag):
    while True:
        image_batch = []
        steerings_batch = []
        
        for i in range(batch_size):
            index = random.randint(0,len(image_paths)-1)
            if training_flag:
                img, steering = augment_image(image_paths[index],steerings[index])
            else:
                img = npimg.imread(image_paths[index])
                steering = steerings[index]
            img = img_preprocess(img)
            image_batch.append(img)
            steerings_batch.append(steering)
        yield(np.asarray(image_batch),np.asarray(steerings_batch))

In [ ]:
def nvidia_model():
    model = Sequential()
    
    model.add(Convolution2D(23, (5,5),(2,2),input_shape=(66,200,3),activation='elu'))
    model.add(Convolution2D(36, (5,5),(2,2),activation='elu'))
    model.add(Convolution2D(48, (5,5),(2,2),activation='elu'))
    model.add(Convolution2D(64, (3,3),(1,1),activation='elu'))
    model.add(Convolution2D(64, (3,3),(1,1),activation='elu'))
    model.add(Flatten())
    model.add(Dense(100,activation='elu'))
    model.add(Dropout(0.2))
    model.add(Dense(50,activation='elu'))
    model.add(Dropout(0.2))
    model.add(Dense(10,activation='elu'))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    optimizer = Adam(learning_rate=1e-3)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mse'])
    
    return model

In [ ]:
model = nvidia_model()
print(model.summary())
checkpoint_filepath = './checkpoint'
model_checkpoint_callback = tensorflow.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_freq = "epoch",
    save_best_only=True)
try:
    shutil.rmtree("/kaggle/working")
except Exception as e: 
    print(e)

In [ ]:
history = model.fit(generate_batch(X_train,Y_train,100,1), steps_per_epoch=300, epochs=30,
                   validation_data=generate_batch(X_valid, Y_valid,10,0),validation_steps=200,  callbacks=[model_checkpoint_callback])

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['training', 'validation'])
plt.title('Loss')
plt.xlabel('Epoch')
model.load_weights(checkpoint_filepath)

In [ ]:
from tensorflow.keras.models import save_model
model.save('./model.h5',save_format='h5')

In [ ]:
 import os
 os.chdir(r'/kaggle/working')
 from IPython.display import FileLink
 FileLink(r'model.h5')

In [ ]:
import setuptools
setuptools.__version__ 